In [60]:
# start here
import numpy as np
import pandas as pd
import math
import csv 

In [2]:
X1 = np.load('X1.npy', allow_pickle=True)
X5 = np.load('X5.npy', allow_pickle=True)
X10 = np.load('X10.npy', allow_pickle=True)

In [3]:
array_train = np.load('data_train.npy', allow_pickle=True)
array_test = np.load('data_test.npy', allow_pickle=True)

In [5]:
y_test = np.load('y_test.npy', allow_pickle=True)
y_train = np.load('y_train.npy', allow_pickle=True)

In [6]:
print(y_test[0])
print(y_train[0])

print(array_test[0])

['normal.']
['normal.']
[0 1 1 1 181 5450 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 8 8 0.0 0.0 0.0 0.0 1.0
 0.0 0.0 9 9 1.0 0.0 0.11 0.0 0.0 0.0 0.0 0.0 'normal.']


In [7]:
print(X1)
print(X5)

[[-1898.9651476709369]
 [-1824.969092395094]
 [-1876.9704146134573]
 ...
 [-1770.9658021365224]
 [-1769.968379504113]
 [-1841.9691356351018]]
[[-1898.9651476709369 -3871.079361335353 22.808065599614473
  -486.52074993088377 -38.69435396506157]
 [-1824.969092395094 -571.0817497411065 21.779870167734803
  -490.34687318896437 -40.269815704723044]
 [-1876.9704146134573 532.9192845991215 21.608316149001936
  -488.8318244250962 -43.500761268384515]
 ...
 [-1770.9658021365224 -3360.0814812917342 27.643254336469653
  -430.1923546260509 -224.91647556456007]
 [-1769.968379504113 -1205.080629920597 27.005072657128974
  -431.2855216604713 -222.32832474912223]
 [-1841.9691356351018 -577.0648228503387 28.128848574919424
  -406.92179666942894 -237.1996702466696]]


In [8]:
def get_p(x, mean, var):
    return np.exp(-(x - mean) ** 2 / (2 * var ** 2)) / (math.sqrt(2 * math.pi) * var) 

In [9]:
global data_yes_cnt, data_no_cnt, data_yes_value, data_no_value
data_yes_cnt, data_no_cnt, data_yes_value, data_no_value = {}, {}, {}, {}
global yes_n, no_n, n
yes_n, no_n, n = 0, 0, 0
global mean_yes, var_yes, mean_no, var_no
mean_yes, var_yes, mean_no, var_no = {}, {}, {}, {}

def init():
    global data_yes_cnt, data_no_cnt, data_yes_value, data_no_value
    global yes_n, no_n, n
    global mean_yes, var_yes, mean_no, var_no
    data_yes_cnt, data_no_cnt, data_yes_value, data_no_value = {}, {}, {}, {}
    mean_yes, var_yes, mean_no, var_no = {}, {}, {}, {}
    yes_n, no_n, n = 0, 0, 0
    for i in range(42):
        data_yes_cnt[i] = {}
        data_no_cnt[i] = {}
        data_yes_value[i] = []
        data_no_value[i] = []    
        if i!=1 and i!=2 and i!=3:
            mean_yes[i] = 0
            var_yes[i]  = 0
            mean_no[i]  = 0
            var_no[i]   = 0    

In [32]:
def get_avg(reader, y_reader, k):
    global data_yes_cnt, data_no_cnt, data_yes_value, data_no_value
    global yes_n, no_n, n
    global mean_yes, var_yes, mean_no, var_no
    line = 0
    for row in reader:
        n += 1
        label = y_reader[line]
        line += 1
        if label == "normal.": # yes
            yes_n += 1
            for i in range(k-1):
                val = row[i]
                if not data_yes_cnt[i].get(val):
                    data_yes_cnt[i][val] = 0
                data_yes_cnt[i][val] += 1
                if i!=1 and i!=2 and i!=3:
                    mean_yes[i] += float(val)
        if label != "normal.": # no
            no_n += 1
            for i in range(k-1):
                val = row[i]
                if not data_no_cnt[i].get(val):
                    data_no_cnt[i][val] = 0
                data_no_cnt[i][val] += 1
                if i!=1 and i!=2 and i!=3:
                    mean_no[i] += float(val)

print("avg ok")

avg ok


In [12]:
def get_cov(reader, y_reader, k):
    global data_yes_cnt, data_no_cnt, data_yes_value, data_no_value
    global yes_n, no_n, n
    global mean_yes, var_yes, mean_no, var_no
    line = 0
    for row in reader:
        label = y_reader[line]
        line += 1
        if label == "normal.": # yes
            for i in range(k-1):
                val = row[i]
                if i!=1 and i!=2 and i!=3:
                    var_yes[i] += (float(val) - mean_yes[i])*(float(val) - mean_yes[i])
        if label != "normal.": # no
            for i in range(k-1):
                val = row[i]
                if i!=1 and i!=2 and i!=3:
                    var_no[i] += (float(val) - mean_no[i])*(float(val) - mean_no[i])
print("cov ok")

cov ok


In [21]:
def get_avg_cov(k):
    global data_yes_cnt, data_no_cnt, data_yes_value, data_no_value
    global yes_n, no_n, n
    global mean_yes, var_yes, mean_no, var_no
    for i in range(k-1):
        if i!=1 and i!=2 and i!=3:
            var_yes[i]  = math.sqrt(var_yes[i] / (n - 1))
            var_no[i]   = math.sqrt(var_no[i] / (n - 1))    

In [14]:
def test(reader, y_reader, k):
    global data_yes_cnt, data_no_cnt, data_yes_value, data_no_value
    global yes_n, no_n, n
    global mean_yes, var_yes, mean_no, var_no
    
    ok, no = 0, 0
    predict = []
    TP, FP, FN, TN = 0, 0, 0, 0   
    line = 0
    for row in reader:
        label = y_reader[line]
        line += 1
        p_yes, p_no = yes_n/(yes_n + no_n), no_n/(yes_n + no_n)
        flag = 0
        for i in range(k-1):
            if i!=1 and i!=2 and i!=3:
                if var_yes[i]!=0:
                    p_yes_temp = get_p(row[i], mean_yes[i], var_yes[i])
                else:
                    p_yes_temp = 1
                if var_no[i]!=0:
                    p_no_temp = get_p(row[i], mean_no[i], var_no[i])
                else:
                    p_no_temp = 1
            else:
                if not data_yes_cnt[i].get(row[i]):
                    p_yes_temp = 1.0*1 / (yes_n + 1)
                else: 
                    p_yes_temp = (data_yes_cnt[i][row[i]] + 1) / (yes_n + 1)
                if not data_no_cnt[i].get(row[i]): 
                    flag = 1
                    p_no_temp = 1 / (no_n + 1)
                else: 
                    p_no_temp  = (data_no_cnt[i][row[i]] + 1) / (no_n + 1)
            p_yes *= p_yes_temp
            p_no *= p_no_temp
        if p_yes > p_no:
            if label == "normal.": 
                ok += 1
                TP += 1
            else: 
                no += 1
                FP += 1
        else:
            if label != "normal.": 
                no += 1
                FN += 1
            else: 
                ok += 1
                TN += 1
    
    print(ok, no)
    print(TP, TN, FP, FN)
    precision = TP / (TP + FP)
    accuracy = (TP+TN) / (TP + TN + FP + FN)
    # f1 = (2 * precision * recall) / (precision + recall)
    print("accuracy: ", str(100 * accuracy)+"%")
    print("precision:", str(100 * precision)+"%")
    

In [ ]:
init()
k = 1
get_avg(X1, y_train, k)
get_cov(X1, y_train, k)
get_avg_cov(k)
print(yes_n, no_n)
# test(array_test, y_test, k)

In [24]:
init()
k = 42
get_avg(array_train, y_train, k)
get_cov(array_train, y_train, k)
get_avg_cov(k)
print(yes_n, no_n)
test(array_test, y_test, k)

194643 784006
97278 396743
5 97273 0 396743
accuracy:  19.691065764410826%
precision: 100.0%


In [58]:
init()
k = 5
cnt = 0

n,m = X5.shape
print(n, m)

for row in X5:
    cnt += 1
    print(cnt)
    if cnt > 2:
        break 
    for i in range(m):
        temp = row[0]
        print(temp[i])
        # print(row[0, i])
# get_avg(X5, y_train, k)
# get_cov(X5, y_train, k)
# get_avg_cov(k)
# test(X5, y_test, k)

978649 5
1
-1898.9651476709369
-3871.079361335353
22.808065599614473
-486.52074993088377
-38.69435396506157
2
-1824.969092395094
-571.0817497411065
21.779870167734803
-490.34687318896437
-40.269815704723044
3


In [ ]:
init()
k = 10
get_avg(X10, y_train, k)
get_cov(X10, y_train, k)
get_avg_cov(k)
test(X10, y_test, k)